# Functions for Preprocess, HMM

In [1]:
import numpy as np
###############
#Reading file #
###############
def parse_HMM(file_path):
    with open(file_path) as f:
        # SIZE=1024
        # f.read(SIZE)
        data = f.readline()
        data= data.split()
        # print(data[0], data[1], data[2])
        transaction_length=int(data[0])
        emission_length=int(data[1])
        map_=data[2]
        transaction_matrix=np.zeros((transaction_length,transaction_length))
        emission_matrix=np.zeros((transaction_length,emission_length))
        initial_prob=np.zeros((transaction_length,1))
        data = f.readline()
        data= data.split()
        for i in range(transaction_length):
            initial_prob[i] = float('0'+data[i])
        for i in range(transaction_length):
            data = f.readline()
            data= data.split()
            for j in range(transaction_length):
                transaction_matrix[i][j]=float('0'+data[j])
            k=0
            for j in range(transaction_length, transaction_length+emission_length, 1):
                # print(j)
                emission_matrix[i][k] = float('0'+data[j])
                k+=1
        # half hour for that?
        # str2=".33"str1='0'print(str1+str2)
    return transaction_matrix, emission_matrix, list(map_), initial_prob
def parse_fa(file_path):
    with open(file_path) as f:

        # ignore first line, elim "\n", stitch
        # ignore first line
        line = f.readline()
        data=''

        lines = f.readlines()
        for line in lines:
           data+=line.rstrip() 
    return data.upper()



class HMM:
    # 
    def viterbi(self, seq, transaction_matrix, emission_matrix, map_, initial_prob):

        num_state = transaction_matrix.shape[0]
        len_seq=len(seq)
        print(num_state, len_seq)
        ###############
        #Table initialization #
        ###############
        dp = np.zeros((num_state, len_seq))
        trace_table=np.zeros((num_state, len_seq), dtype=int)
        dp[:, 0]=np.log(initial_prob.reshape(num_state))
        ###############
        #Table generation #
        ###############
        for i in range(1, len_seq):
            for k in range(num_state):
                seq_prob = dp[:, i-1]+np.log(transaction_matrix[:, k])+ \
                    np.log(emission_matrix[k, map_.index(seq[i])])
                trace_table[k, i-1]=np.argmax(seq_prob)
                dp[k, i] = np.max(seq_prob)
        return dp, trace_table
        # using another table to get the result .
    ###############
    #Trace back Part #
    ###############
    def build_trace_path(self, dp, trace_table):
        len_seq=dp.shape[1]
    
        result_table=np.zeros(len_seq,dtype=int)
        last_state = np.argmax(dp[:,-1])
        result_table[len_seq-1]=last_state
        # need to include the 0
        for i in range(len_seq-2, -1, -1):
            # print(i)
            last_state= trace_table[last_state, i]
            result_table[i]=last_state
        return result_table

# Run and output

In [6]:


# if name == "__main__":
# about 28 sec
HMM_PATH="/home/aistudio/work/data/example.hmm"
FA_PATH="/home/aistudio/work/data/example.fa"

transaction_matrix, emission_matrix, map_, initial_prob= parse_HMM(HMM_PATH)
test_genome= parse_fa(FA_PATH)
# print(test_genome)
# print(transaction_matrix, emission_matrix, map_, initial_prob)
# test_genome=test_genome[0:10077]


hmm=HMM()

dp, trace_table= hmm.viterbi(test_genome, transaction_matrix, emission_matrix, map_, initial_prob)
# professor's requiement|
print("### final result of S(k,i):")
print(dp[:,-1])

result_table = hmm.build_trace_path(dp, trace_table)
#output the summary 

i=0 
j=0
total_num_seg=0
total_num_B=0
state_l=["A","B"]
while(j<len(result_table)-1):
    j+=1   
    if(result_table[j]!=result_table[i]):
        # start i, end j-1
        # to make the index start from 1
        print("%r  %r State%r "%(i+1, j-1+1, state_l[result_table[i]]))
        if(result_table[i]==1):
            total_num_B+=1
        total_num_seg+=1
        # update i
        i=j
   
# start i,end j
# to make the index start from 1
print("%r  %r State%r "%(i+1, j+1, state_l[result_table[i]]))
if(result_table[i]==1):
    total_num_B+=1
total_num_seg+=1
print("###  ")
print("%r segments in state B, total segments %r"%(total_num_B, total_num_seg)) 

![](https://ai-studio-static-online.cdn.bcebos.com/7e7d728f28d14df0aa45834a042655b26ba90f48e22048dc964e579b4bf3f839)
![](https://ai-studio-static-online.cdn.bcebos.com/e538b8f200dc4279a12be282c0c796bf4688ce7d99a645a38480bfcbb93d9b0e)
